## 01. 라이브러리 로드

In [164]:
import pandas as pd
import numpy as np

## 02. 데이터 로드 및 컬럼 삭제

In [165]:
df = pd.read_excel("data/krx/daily_close_2011_2024_04.xlsx")
df.drop(columns=['Unnamed: 0','날짜'], inplace=True)

## 03. 초과수익률 계산
- 수익률(%) : 거래소코드별 계산
- 코스피_코스닥_수익률 데이터 로드, 컬럼명 재정의 후 결합
- 코스피상장종목, 코스닥상장종목 데이터 로드, 컬럼명 재정의
- 초과수익률 계산 (calculate_excess_return_final 함수 사용)

In [166]:
pct_result = df.groupby('거래소코드')['수정종가'].pct_change()
df['수익률(%)'] = pct_result.groupby(df['거래소코드']).shift(-1) * 100
df_return = df.sort_values(by=['거래소코드', '회계년도'])

#### 코스피_코스닥 수익률 데이터 로드

In [167]:
kospi_kosdaq = pd.read_excel('data/krx/코스피_코스닥_수익률_4월.xlsx')
kospi_kosdaq.rename(columns={'연도': '회계년도', 'KOSPI 수익률 (%)': '코스피수익률', 'KOSDAQ 수익률 (%)': '코스닥수익률'}, inplace=True)

#### 데이터 병합 - 수익률 df와 코스피_코스닥 수익률 df

In [168]:
df_merged_kk = pd.merge(df_return, kospi_kosdaq, on=['회계년도'], how='left')

#### 코스피, 코스당 상장종목 데이터 로드

In [169]:
kospi_list = pd.read_excel('data/krx/코스피상장종목.xlsx')
kosdaq_list = pd.read_excel('data/krx/코스닥상장종목.xlsx')

#### 컬럼명 재정의 및 소속코드 컬럼 제거

In [170]:
kospi_list['거래소코드'] = kospi_list['거래소코드'].astype(str).str.zfill(6)
kospi_list['회계년도'] = kospi_list['회계년도'].str.split('/').str[0].astype(int)

kosdaq_list['거래소코드'] = kosdaq_list['거래소코드'].astype(str).str.zfill(6)
kosdaq_list['회계년도'] = kosdaq_list['회계년도'].str.split('/').str[0].astype(int)

kospi_list.drop(columns=['소속코드'], inplace=True)
kosdaq_list.drop(columns=['소속코드'], inplace=True)

#### kospi_list와 df_merged_kk 병합
- 소속코드 = 1 -> 코스피
- 수정종가, 수익률(%), 코스피수익률, 코스닥수익률 컬럼 제거

In [171]:
merged_kospi = pd.merge(kospi_list, df_merged_kk, on=['회계년도', '거래소코드'], how='left')
merged_kospi['소속코드'] = 1
merged_kospi.drop(columns=['수정종가', '코스피수익률', '코스닥수익률'], inplace=True)

#### kosdaq_list와 df_merged_kk 병합
- 수정종가, 수익률(%), 코스피수익률, 코스닥수익률 컬럼 제거

In [172]:
merged_kosdaq = pd.merge(kosdaq_list, df_merged_kk, on=['회계년도', '거래소코드'], how='left')
merged_kosdaq.drop(columns=['수정종가', '코스피수익률', '코스닥수익률'], inplace=True)

#### merged_kk에 merged_kospi, merged_kosdaq 병합
- 회사명 컬럼 결측치 채우기
- 회사명x, 회사명y 컬럼 제거
- 소속코드 결측치 채우기

In [175]:
merged_1 = pd.merge(df_merged_kk, merged_kospi, on=['회계년도', '거래소코드'], how='left')
merged_2 = pd.merge(merged_1, merged_kosdaq, on=['회계년도', '거래소코드'], how='left')
merged_2['회사명'] = merged_2['회사명_x'].fillna(merged_2['회사명_y'])
merged_2['수익률(%)'] = merged_2['수익률(%)_x'].fillna(merged_2['수익률(%)_y'])
merged_2.drop(columns=['회사명_x', '회사명_y'], inplace=True)
merged_2.drop(columns=['수익률(%)_x', '수익률(%)_y'], inplace=True)
merged_2['소속코드'].fillna(5, inplace=True)

C:\Users\tmddn\AppData\Local\Temp\ipykernel_17520\2118040445.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_2['소속코드'].fillna(5, inplace=True)


#### merged_kk 수익률(%), 회사명 컬럼 결측치 제거

In [178]:
merged_2 = merged_2[merged_2['수익률(%)'].notna()]
merged_2 = merged_2[merged_2['회사명'].notna()]

#### 초과수익률 계산 함수 정의
- 소속코드 : 5 -> 코스닥, 소속코드 : 1 -> 코스피

In [180]:
def calculate_excess_return_final(row):
    if row['소속코드'] == 5:  # 코스닥
        return row['수익률(%)'] - row['코스닥수익률']
    elif row['소속코드'] == 1:  # 코스피
        return row['수익률(%)'] - row['코스피수익률']
    return None 

merged_2['초과수익률'] = merged_2.apply(calculate_excess_return_final, axis=1)

## 05. 데이터저장
- 초과수익률 컬럼 결측치 제거

In [182]:
merged_2 = merged_2[merged_2['초과수익률'].notna()]
df_final = merged_2[['회사명', '거래소코드', '회계년도', '수정종가', '초과수익률', '수익률(%)']].copy()
df_final = df_final.sort_values(by=['회사명', '회계년도'])
df_final.to_excel('data/krx/초과수익률.xlsx', index=False)